![Author](https://img.shields.io/badge/Author-Soufiane%20AAZIZI-brightgreen)
[![Medium](https://img.shields.io/badge/Medium-Follow%20Me-blue)](https://medium.com/@aazizi.soufiane)
[![GitHub](https://img.shields.io/badge/GitHub-Follow%20Me-lightgrey)](https://github.com/aazizisoufiane)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-Connect%20with%20Me-informational)](https://www.linkedin.com/in/soufiane-aazizi-phd-a502829/)

---
# Streamlining Machine Learning Model Deployment with CI/CD and MLOps

In today's ever-evolving landscape of data science and machine learning, proficiency in managing and deploying machine learning models has transitioned from a desirable skill to an absolute necessity. Aspiring data scientists are no longer solely responsible for building sophisticated models; they are also expected to seamlessly integrate these models into real-world applications. This is where the convergence of Continuous Integration and Continuous Deployment (CI/CD) and Machine Learning Operations (MLOps) takes center stage.

In this Jupyter Notebook, we embark on an exciting journey into the realm of CI/CD and MLOps. Our goal is to demystify these crucial concepts and underscore their significance for emerging data scientists. Through hands-on exploration, we will delve into a project that harnesses the power of Amazon SageMaker and AWS Step Functions to simplify the entire process of data preprocessing, model training, and deployment. This project serves as an invaluable stepping stone for data scientists eager to acquire essential CI/CD and MLOps skills, further enhancing their appeal in today's competitive job market.

Join us on this educational journey as we unlock the potential of CI/CD and MLOps in the context of machine learning, empowering you to take your data science expertise to the next level.
## 

## Table of Contents
- [Installing Required Packages](#Installing)
- [Imports](#imports)
- [Initialization](#initialization)
- [Data Preprocessing Step](#data-preprocessing)
- [Model Training and Evaluation Configuration ](#model-training)
- [Model Deployment Configuration ¶](#Endpoint-config) 
- [Workflow Creation and Configuration](#Create-WorkFlow)
- [Workflow Execution](#Execute-WorkFlow) 
- [Model Inference](#prediction) 

### Installing Required Packages <a class="anchor" id="Installin"></a>
Before we begin, let's make sure we have all the necessary Python packages installed. You can run the following command to install any missing packages:


In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install stepfunctions  omegaconf  nb-black

In [ ]:
! pip install -U sagemaker

### Auto-formatting and Auto-Reloading Configuration

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

## Imports <a class="anchor" id="imports"></a>

In [ ]:
import os
import time
import boto3
import sagemaker
import stepfunctions
from config import config
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.huggingface import HuggingFace
from stepfunctions import steps
from botocore.config import Config
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow
from stepfunctions.steps import (
    Chain,
    ChoiceRule,
    ModelStep,
    ProcessingStep,
    TrainingStep,
    TransformStep,
    Parallel,
)
from sagemaker.processing import ProcessingInput, ProcessingOutput
from dotenv import load_dotenv, find_dotenv

# Initialization <a class="anchor" id="initialization"></a>
- Set up environment, load data, and configure settings.

In [ ]:
# Load environment variables from a .env file
_ = load_dotenv(find_dotenv())

# Retrieve the SageMaker workflow execution role from environment variables
workflow_execution_role = os.getenv("SAGEMAKER_WORKFLOW_ROLE")

In [ ]:
execution_input = ExecutionInput(
    schema={
        "JobName": str,
        "PreprocessingJobName": str,
        "TrainingJobName": str,
        "ModelName": str,
        "EndpointName": str,
    }
)

In [ ]:
# Define a Fail state for handling SageMaker processing job failures
failed_state_sagemaker_processing_failure = stepfunctions.steps.states.Fail(
    "ML Workflow failed", cause="SageMakerProcessingJobFailed"
)

# Define a Catch state for capturing specific errors and transitioning to the Fail state
catch_state_processing = stepfunctions.steps.states.Catch(
    error_equals=["States.TaskFailed"],
    next_step=failed_state_sagemaker_processing_failure,
)

In [ ]:
failed_state_sagemaker_training_failure = stepfunctions.steps.states.Fail(
    "ML Training failed", cause="SageMakerTrainingJobFailed"
)

catch_state_training = stepfunctions.steps.states.Catch(
    error_equals=["States.TaskFailed"],
    next_step=failed_state_sagemaker_training_failure,
)

In [ ]:
failed_state_sagemaker_training_failure = stepfunctions.steps.states.Fail(
    "ML Save Model failed", cause="SageMakerSaveModelJobFailed"
)

catch_state_save_model = stepfunctions.steps.states.Catch(
    error_equals=["States.TaskFailed"],
    next_step=failed_state_sagemaker_training_failure,
)

In [ ]:
failed_state_sagemaker_inference_failure = stepfunctions.steps.states.Fail(
    "ML Inference failed", cause="SageMakerInferenceFailed"
)

catch_state_inference = stepfunctions.steps.states.Catch(
    error_equals=["States.TaskFailed"],
    next_step=failed_state_sagemaker_inference_failure,
)

### Sagemaker Configuration

In [ ]:
# Create a SageMaker Boto3 client with custom configuration
sm_boto = boto3.client(
    "sagemaker",
    config=Config(connect_timeout=5, read_timeout=60, retries={"max_attempts": 60}),
)

# Initialize a SageMaker session using the custom Boto3 client
sagemaker_session = sagemaker.Session(sagemaker_client=sm_boto)

# Retrieve the AWS region from the SageMaker session
region = sagemaker_session.boto_session.region_name

# Get the execution role required for SageMaker operations
role = get_execution_role()

# Retrieve configuration values for the S3 bucket and prefix
bucket = config.s3.bucket
prefix = config.s3.prefix
s3_bucket_base_uri = config.s3.s3_bucket_base_uri

### Functions

In [ ]:
def job_name(jobname):
    """
    Generate a unique job name for an Amazon SageMaker job based on a given 'jobname' and current timestamp.

    Args:
        jobname (str): A descriptive name for the job.

    Returns:
        str: A unique job name incorporating the 'jobname' and timestamp.
    """
    return f"MultiLabelClassification-{jobname}--{time.strftime('%Y%m%d%H%M%S', time.gmtime())}"


def upload_code(bucket_name, prefix_name, script_location):
    """
    Upload code or script to an Amazon S3 bucket for use in SageMaker.

    Args:
        bucket_name (str): The name of the S3 bucket where the code will be uploaded.
        prefix_name (str): The prefix or directory within the S3 bucket where the code will be stored.
        script_location (str): The local path to the code or script file.

    Returns:
        str: The S3 URI of the uploaded code.
    """
    return sagemaker_session.upload_data(
        script_location,
        bucket=bucket_name,
        key_prefix=f"{prefix_name}/{script_location}",
    )

# Data Preprocessing Step <a class="anchor" id="data-preprocessing"></a>

In this section of the notebook, our focus shifts towards the preparation and definition of a SageMaker processing step that is responsible for data preprocessing. Data preprocessing plays a pivotal role in any machine learning workflow as it encompasses the tasks of cleaning, transforming, and organizing raw data into a format that is suitable for model training.

### Key Components

1. **Preprocessing Step Name**: We initiate this process by generating a descriptive name for the preprocessing step, which includes a timestamp. This unique identifier aids in tracking and managing this step within the workflow.

2. **SKLearnProcessor Configuration**: We define a function called `sklearn_processor` that is responsible for creating an instance of the SageMaker SKLearnProcessor. This processor is a fundamental component of SageMaker, allowing us to run data preprocessing jobs using scikit-learn scripts. The configuration includes essential details such as the framework version, the IAM role required for job execution, the instance type for computation, and the number of instances.

3. **Processing Step**: Here, we proceed to define the actual SageMaker processing step dedicated to data preprocessing. This step represents the execution of the preprocessing job within our workflow and includes the following key elements:
   - `preprocessing_step_name`: The name generated earlier for this step.
   - `processor`: An instance of the SKLearnProcessor, created using the `sklearn_processor` function.
   - `job_name`: The name of the SageMaker processing job, extracted from an `execution_input` dictionary. It serves as a crucial identifier for tracking job progress.
   - `inputs`: Configuration specifying the location of input data for preprocessing.
   - `outputs`: Configuration indicating where the preprocessed data will be stored.
   - `container_entrypoint`: The entry point for the processing job's container, specifying the script to be executed.
   - `container_arguments`: Additional arguments passed to the processing container, which may include parameters like the train-test split ratio.

The primary objective of this section is to establish a SageMaker processing step tailored to handle data preprocessing tasks within the broader machine learning workflow. By configuring various aspects, we ensure that the data undergoes proper preprocessing and is ready for subsequent stages of model training.

In [ ]:
# Define the location of the preprocessing script within the project
PREPROCESSING_SCRIPT_LOCATION = "preprocess/code"

# Create the output path for preprocessing results using the S3 bucket base URI and prefix
output_preprocess = "{}/{}".format(s3_bucket_base_uri, config.s3.prefix)

In [ ]:
# Upload the preprocessing script to an S3 location and get the S3 URI
input_code_preprocess = sagemaker_session.upload_data(
    PREPROCESSING_SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=f"{prefix}/{PREPROCESSING_SCRIPT_LOCATION}",
)

In [ ]:
# Define a list of ProcessingInput objects for the preprocessing job
inputs_preprocess = [
    ProcessingInput(
        source=f"{config.s3.s3_bucket_base_uri}/{config.s3.input}",
        destination="/opt/ml/processing/input",
        input_name="input-data",
    ),
    ProcessingInput(
        source=input_code_preprocess,
        destination="/opt/ml/processing/input/code",
        input_name="code",
    ),
    ProcessingInput(
        source=f"s3://{bucket}/{prefix}/{PREPROCESSING_SCRIPT_LOCATION}/config",
        destination="/opt/ml/processing/input/config",
        input_name="code-config",
    ),
]

In [ ]:
# Define a list of ProcessingOutput objects for the preprocessing job
outputs_preprocess = [
    ProcessingOutput(
        source="/opt/ml/processing/train",
        destination=output_preprocess,
        output_name="train_data",
    ),
    ProcessingOutput(
        source="/opt/ml/processing/test",
        destination=output_preprocess,
        output_name="test_data",
    ),
    ProcessingOutput(
        source="/opt/ml/processing/labels",
        destination=output_preprocess,
        output_name="labels_data",
    ),
]

In [ ]:
# Create a descriptive name for the preprocessing step, including a timestamp.
preprocessing_step_name = f"Multilabel Classification - Preprocessing Step {time.strftime('%Y%m%d%H%M%S', time.gmtime())}"


# Define an Amazon SageMaker SKLearnProcessor with custom settings.
def sklearn_processor(instance_type="ml.m5.xlarge"):
    """
    Create an SKLearnProcessor instance for Amazon SageMaker processing jobs.

    Args:
        instance_type (str): The Amazon SageMaker instance type for processing jobs.

    Returns:
        sagemaker.processing.SKLearnProcessor: An instance of the SKLearnProcessor.
    """
    return SKLearnProcessor(
        framework_version="1.2-1",
        role=role,  # Ensure the 'role' variable is defined and appropriate.
        instance_type=instance_type,
        instance_count=1,
        # max_runtime_in_seconds=1200,  # Uncomment and customize if needed.
    )


# Define the SageMaker processing step for data preprocessing.
processing_step = ProcessingStep(
    preprocessing_step_name,
    processor=sklearn_processor(),
    job_name=execution_input["PreprocessingJobName"],
    inputs=inputs_preprocess,  # Define your input data configuration here.
    outputs=outputs_preprocess,  # Define your output data configuration here.
    container_entrypoint=["python3", "/opt/ml/processing/input/code/run.py"],
    container_arguments=[
        "--train-test-split-ratio",
        "0.2",
    ],  # Uncomment and customize if needed.
)

# Model Training and Evaluation Configuration <a class="anchor" id="model-training"></a>

In this section, we configure various components essential for the model training and evaluation phase of our machine learning workflow.

### Metric Definitions
To assess the performance of our trained model, we define a list of metric definitions. These metrics serve as key indicators of the model's quality and effectiveness during evaluation. The metrics include metrics like loss, accuracy, F1 score, ROC (Receiver Operating Characteristic), ROC AUC (Area Under the Curve), precision, recall, runtime, samples per second, and epoch. These metrics will be computed and tracked during model training.

### Instance Volume Configuration
Different Amazon SageMaker instance types come with varying storage capacities. To ensure that our model training jobs have adequate storage space, we define an `instance_volume` configuration. This mapping specifies the instance type and the associated volume size, ensuring that the chosen instance type has sufficient storage capacity for the training process.

### Training Parameters
We set various training parameters necessary for configuring the HuggingFace estimator, which includes:
- `epochs`: The number of training epochs.
- `train-batch-size`: The batch size for training.
- `eval_steps`: The number of evaluation steps.
- `instance_type`: The SageMaker instance type to use for training.
- `volume_size`: The volume size for the SageMaker instance.

### Preprocessed Data Output Configuration
Before moving forward, we retrieve the output configuration from the previous preprocessing step. This configuration contains information about the locations where preprocessed data is stored. Specifically, we identify the S3 URIs for preprocessed training, test, and labels data.

### Model Estimator Generation
We define a function called `generate_estimator()` responsible for creating the HuggingFace estimator. The estimator is configured with parameters such as the model name, number of epochs, batch size, output directory for checkpoints, evaluation steps, instance type, and other necessary details. This estimator serves as the foundation for model training.

### Data Preparation
We define another function named `generate_data()` that prepares the data for model training. This function specifies the paths to preprocessed training, test, and labels data and creates a dictionary specifying the data sources and content types.

### Training Step Generation
With the groundwork laid, we use the `generate_training_step()` function to create a SageMaker TrainingStep in our Step Functions workflow. This step includes essential information such as the job name, estimator, data sources, and other configurations. It represents the process of training our machine learning model.

### Model Deployment Step
Finally, we create a ModelStep named "Save model" that saves the trained model. This step uses the model obtained from the training step, specifies the model name, and defines the SageMaker instance type for model deployment.

These configurations and steps prepare us for the subsequent phases of our machine learning workflow, including model training and evaluation, and model deployment.


In [ ]:
# Define a list of metric definitions for model evaluation
metric_definitions = [
    {"Name": "eval_loss", "Regex": "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_accuracy", "Regex": "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_f1", "Regex": "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_roc", "Regex": "'eval_roc': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_roc_auc", "Regex": "'eval_roc_auc': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_precision", "Regex": "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_recall", "Regex": "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_runtime", "Regex": "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {
        "Name": "eval_samples_per_second",
        "Regex": "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?",
    },
    {"Name": "epoch", "Regex": "'epoch': ([0-9]+(.|e\-)[0-9]+),?"},
]

# Define instance volume configurations based on instance type
instance_volume = {
    "ml.g4dn.16xlarge": 900,
    "ml.g4dn.8xlarge": 500,
    "ml.g4dn.4xlarge": 225,
    "ml.g4dn.2xlarge": 225,
    "ml.g4dn.xlarge": 125,
}

# Define training parameters such as epochs, batch size, evaluation steps, instance type, and volume size
params = {
    "epochs": 1,
    "train-batch-size": 8,
    "eval_steps": 1,
    "instance_type": "ml.g4dn.4xlarge",
    "volume_size": 125,
}

In [ ]:
# Extract the ProcessingOutputConfig from the processing step's parameters
output_config = processing_step.fields["parameters"]["ProcessingOutputConfig"]

# Initialize variables to store the S3 URIs for preprocessed data
preprocessed_training_data = None
preprocessed_test_data = None
preprocessed_labels_data = None

# Iterate through the list of outputs in the ProcessingOutputConfig
for output in output_config["Outputs"]:
    if output["OutputName"] == "train_data":
        # Set the S3 URI for preprocessed training data
        preprocessed_training_data = os.path.join(
            output["S3Output"]["S3Uri"], "train.csv"
        )
    if output["OutputName"] == "test_data":
        # Set the S3 URI for preprocessed test data
        preprocessed_test_data = os.path.join(output["S3Output"]["S3Uri"], "test.csv")
    if output["OutputName"] == "labels_data":
        # Set the S3 URI for preprocessed labels data
        preprocessed_labels_data = os.path.join(
            output["S3Output"]["S3Uri"], "labels.csv"
        )

In [ ]:
def generate_estimator():
    # Define the S3 URIs for checkpoints and training jobs
    checkpoint_s3_uri = f"s3://{config.s3.bucket}/{config.s3.prefix}/checkpoints"
    output_path = f"s3://{config.s3.bucket}/{config.s3.prefix}/training_jobs"

    # Define hyperparameters for the HuggingFace estimator
    hyperparameters = {
        "model_name": "distilbert-base-uncased",
        "epochs": params["epochs"],
        "train-batch-size": params["train-batch-size"],
        "output_dir": checkpoint_s3_uri,  # Use the checkpoint S3 URI for output
        "eval_steps": params["eval_steps"],
    }

    # Create and configure the HuggingFace estimator for SageMaker
    return HuggingFace(
        entry_point="train.py",  # Entry point script for training
        source_dir="train/code",  # Source directory containing training code
        output_path=f"{output_path}/",  # Output path for storing model artifacts
        code_location=output_path,
        role=role,  # SageMaker execution role
        base_job_name=f"multi-label-classification",  # Base job name for SageMaker job
        checkpoint_s3_uri=checkpoint_s3_uri,  # Specify the checkpoint input path
        instance_type=params["instance_type"],  # SageMaker instance type for training
        instance_count=1,  # Number of training instances
        transformers_version="4.6",
        pytorch_version="1.7",
        py_version="py36",
        hyperparameters=hyperparameters,  # Hyperparameters for training
        metric_definitions=metric_definitions,  # Metric definitions for evaluation
        volume_size=instance_volume[params["instance_type"]],  # Instance volume size
        sagemaker_session=sagemaker_session,  # SageMaker session
    )

In [ ]:
def generate_data():
    # Define the paths to preprocessed training, test, and labels data
    train_path = preprocessed_training_data
    test_path = preprocessed_test_data
    labels_path = preprocessed_labels_data

    # Create a dictionary specifying the training data sources and content types
    data = {
        "train": sagemaker.TrainingInput(train_path, content_type="text/libsvm"),
        "test": sagemaker.TrainingInput(test_path, content_type="text/libsvm"),
        "labels": sagemaker.TrainingInput(labels_path, content_type="text/libsvm"),
    }

    return data


def generate_training_step(instance="ml.g4dn.2xlarge"):
    # Generate a unique job name based on the current timestamp
    jobname = (
        f"multi-label-classification--{time.strftime('%Y%m%d%H%M%S', time.gmtime())}"
    )

    # Create a TrainingStep in the Step Functions workflow
    training_step = steps.TrainingStep(
        f"Trainning -- instance {instance}",  # Step name
        estimator=generate_estimator(),  # Use the HuggingFace estimator
        data=generate_data(),  # Specify training data sources
        job_name=jobname,  # Job name for SageMaker training job
        wait_for_completion=True,  # Wait for training job to complete
    )

    return training_step, jobname

In [ ]:
# Generate a SageMaker TrainingStep using the 'generate_training_step' function
training_step, training_job_name = generate_training_step(params["instance_type"])

# Create a ModelStep to save the trained model
model_step = steps.ModelStep(
    "Save model",  # Step name
    model=training_step.get_expected_model(),  # Use the model from the training step
    model_name=training_job_name,  # Specify the model name
    instance_type=params[
        "instance_type"
    ],  # SageMaker instance type for model deployment
)


# Model Deployment Configuration <a class="anchor" id="Endpoint-config"></a>

In this section, we configure the deployment of our trained machine learning model as a SageMaker endpoint, making it accessible for inference and predictions.

### Endpoint Configuration Step
We begin by defining an `EndpointConfigStep` named "Create Endpoint Config." This step is responsible for creating an endpoint configuration that specifies the deployment details for our trained model. The key configurations include:
- `endpoint_config_name`: The name for the endpoint configuration, which is set to the `training_job_name` from the training step. This ensures consistency between the model and endpoint configurations.
- `model_name`: The name of the model to associate with this endpoint configuration, also set to the `training_job_name`.
- `initial_instance_count`: The initial number of instances (1 in this case) for the endpoint.
- `instance_type`: The SageMaker instance type to use for the endpoint (e.g., "ml.m5.large").

### Endpoint Step
Following the creation of the endpoint configuration, we define an `EndpointStep` named "Create Endpoint." This step is responsible for the actual deployment of the model as an endpoint. Key configurations for this step include:
- `endpoint_name`: The name of the endpoint, which is set to the `training_job_name`. This ensures a consistent naming convention.
- `endpoint_config_name`: The name of the endpoint configuration, which is also set to the `training_job_name`.

These two steps together facilitate the deployment of our trained machine learning model as an accessible SageMaker endpoint, ready to serve predictions and inferences.


In [ ]:
# Define an Endpoint Configuration Step to create an endpoint configuration.
endpoint_config_step = steps.EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=training_job_name,  # Use the training job name as the endpoint config name
    model_name=training_job_name,  # Use the training job name as the model name
    initial_instance_count=1,
    instance_type="ml.m5.large",
)

# Define an Endpoint Step to create the actual endpoint.
endpoint_step = steps.EndpointStep(
    "Create Endpoint",
    endpoint_name=training_job_name,  # Use the training job name as the endpoint name
    endpoint_config_name=training_job_name,  # Use the training job name as the endpoint config name
)


# Workflow Creation and Configuration <a class="anchor" id="Create-WorkFlow"></a>

In this section, we create and configure a Step Functions workflow named 'MlOpsWorkflow' to orchestrate the entire machine learning operations (MLOps) process, including data preprocessing, training, model deployment, and endpoint setup. The workflow is defined by chaining various steps together to ensure a streamlined execution.

### Catch States for Error Handling
Before defining the workflow, we set up catch states for error handling. These catch states are designed to handle failures that might occur during different stages of the workflow. The catch states include:
- `catch_state_processing`: Handling errors related to data processing.
- `catch_state_training`: Handling errors during model training.
- `catch_state_save_model`: Handling errors when saving the trained model.
- `catch_state_inference`: Handling errors during model inference and predictions.

### Workflow Graph Definition
The core of the workflow is defined in a `workflow_graph` by chaining together multiple steps. These steps include:
- `processing_step`: Responsible for data preprocessing.
- `training_step`: Initiates model training.
- `model_step`: Saves the trained model.
- `endpoint_config_step`: Creates an endpoint configuration.
- `endpoint_step`: Deploys the model as an endpoint.

The steps are executed sequentially, ensuring that each step is completed successfully before moving on to the next. The catch states are attached to these steps to handle any potential failures gracefully.

### Workflow Creation
After defining the workflow graph and configuring error handling, we create the Step Functions workflow named 'MlOpsWorkflow.' The configuration for this workflow includes:
- `name`: The name of the workflow, set to "MlOpsWorkflow" for clarity.
- `definition`: The workflow graph we defined earlier, serving as the workflow's definition.
- `role`: The execution role required for the workflow, specified as `workflow_execution_role`.

Once created, this workflow can be executed to automate the entire MLOps pipeline, ensuring a smooth and reliable process for training and deploying machine learning models.


In [ ]:
from stepfunctions.workflow import Workflow

In [ ]:
# Add catch states to handle failures for processing, training, and model steps
processing_step.add_catch(catch_state_processing)
training_step.add_catch(catch_state_training)
model_step.add_catch(catch_state_save_model)
endpoint_step.add_catch(catch_state_inference)

In [ ]:
# Create a workflow graph by chaining the processing, training, model, endpoint config, and endpoint steps
workflow_graph = Chain(
    [processing_step, training_step, model_step, endpoint_config_step, endpoint_step]
)

In [ ]:
# Create a Step Functions workflow named 'MlOpsWorkflow' using the defined graph and execution role
branching_workflow = Workflow(
    name=job_name("MlOpsWorkflow"),  # Specify the workflow name
    definition=workflow_graph,  # Use the workflow graph as the definition
    role=workflow_execution_role,  # Specify the execution role
)

In [ ]:
# Create the Step Functions workflow
branching_workflow.create()

# Workflow Execution <a class="anchor" id="Execute-WorkFlow"></a>

In this section, we execute the previously defined Step Functions workflow named 'MlOpsWorkflow.' The execution is initiated with specific input parameters that customize the execution process. These parameters include:
- `"PreprocessingJobName"`: A parameter specifying the job name for the data preprocessing step.
- `"TrainingJobName"`: A parameter specifying the job name for the model training step.

### Initiating Workflow Execution
To kick off the workflow execution, we use the `execute` method on the `branching_workflow` object. This method starts the execution of the workflow with the provided input parameters. The input parameters allow us to configure and customize the execution for each run. Additional input parameters can be included as needed to further tailor the workflow execution.

### Retrieving Workflow Output
After initiating the workflow execution, we retrieve the output of the execution. Note that the `get_output` method is called with the `wait=False` parameter, indicating that we do not want to block the execution and wait for its completion. This is particularly useful for non-blocking executions, and it allows us to continue with other tasks or processes while monitoring the progress of the workflow.

The output of the execution can include various pieces of information, such as the status of the workflow, step execution details, and any error messages encountered during execution.

By following this approach, you can automate and manage complex machine learning workflows using Step Functions while having the flexibility to provide input parameters for each execution, ensuring efficient and customized MLOps processes.


In [ ]:
# Execute the Step Functions workflow with input parameters
execution = branching_workflow.execute(
    inputs={
        "PreprocessingJobName": job_name("PreprocessingJobName"),
        "TrainingJobName": job_name("TrainingJobName"),
        # Additional input parameters can be provided here if needed
    }
)

In [ ]:
# Get the output of the workflow execution (non-blocking, does not wait for completion)
execution.get_output(wait=False)

# Model Inference <a class="anchor" id="prediction"></a>

In this section, we perform inference on a deployed machine learning model using SageMaker. The steps involved include creating a predictor, defining input data, creating a payload, making predictions, and printing the results.

### Creating a Predictor
To perform inference, we first create a SageMaker predictor using the `HuggingFacePredictor` class. The predictor is configured with specific settings, including the SageMaker endpoint name (`endpoint_name`) and the SageMaker session (`sagemaker_session`). The endpoint name indicates which deployed model to use for inference.

### Defining Input Data
We define the input data for inference as a list of strings. In this case, the input data is a list of text strings that we want to use for making predictions. This data can vary depending on the use case and requirements.

### Creating the Payload
To send a request for inference, we create a payload, which is formatted as a JSON object. The payload includes two key-value pairs:
- `"inputs"`: This key holds the input data defined earlier as a list of strings.
- `"parameters"`: Additional parameters that can be customized based on the inference requirements. In this example, we specify `{"return_all_scores": True}` as a parameter.

### Making Predictions
With the payload ready, we use the `predict` method of the predictor to send the request for inference. The predictor communicates with the specified SageMaker endpoint and returns the inference results.

### Printing Predictions
Finally, we print the predictions obtained from the model inference. The format and content of the predictions may vary depending on the model and application. In this example, the predictions are printed to the console for review and further processing.

This section demonstrates how to perform real-time model inference with SageMaker, making it possible to use machine learning models for various applications, such as natural language processing, image classification, and more.


In [ ]:
from sagemaker.huggingface import HuggingFacePredictor

# Create a predictor for your SageMaker endpoint
predictor = HuggingFacePredictor(
    endpoint_name="multi-label-classification--20230922090633",
    sagemaker_session=sagemaker_session,
)

# Define the input data as a list of strings
inputs = ["Streamlining Machine Learning Model Deployment with CI/CD and MLOps"]

# Create the payload as a JSON object
payload = {
    "inputs": inputs,
    "parameters": {"return_all_scores": True},
}

# Make predictions
predictions = predictor.predict(payload)

# Print the predictions
print(predictions)
